EM Topic models The UCI Machine Learning dataset repository hosts several datasets recording word counts for documents here. You will use the NIPS dataset.
Cluster this to 30 topics, using a simple mixture of multinomial topic model, as lectured in class.
Produce a graph showing, for each topic, the probability with which the topic is selected.
Produce a table showing, for each topic, the 10 words with the highest probability for that topic.

In [ ]:
import numpy as np
import scipy
import pandas as pd
import sklearn.cluster as sc
import matplotlib.pyplot as plt
import seaborn

In [ ]:
D = 1500
W = 12419

doc_word = np.ones((D, W))

# Read doc id and word id's into a matrix
doc = open('docword.nips.txt')
lines = doc.readlines()
for line in lines[3:]:
    curr_vals = list(map(int, line.split()))
    doc_word[curr_vals[0]-1, curr_vals[1]-1] += curr_vals[2]

print("finished creating matrix")

In [ ]:
#Read in vocab
vocab = []
with open('vocab.nips.txt') as f:
    for line in f:
        vocab.append(line.strip())

In [ ]:
# Generate initial pi and p vectors
T = 30
w = np.zeros((D, T))
pi = np.ones(T)/T
P = np.random.random_sample((T,W))

for i in range(T):
    P[i, :] /= np.sum(P[i, :]) # Normalize

print(P)
print(pi)

In [ ]:
i = 0
j = 0
top = doc_word[i,:] * np.log(P[j,:])
np.sum(top)

In [ ]:
def em_step(x, w, p, pi):
    # Find the max number
    
    # For each document
    for i in range(D):
        temp = np.zeros(T)
        # For each topic
        for l in range(T):
            # Calculate new pi values
            temp[l] = np.sum(x[i,:] * np.log(p[l,:])) + np.log(pi[l])        
            max_num = np.amax(temp)
        bottom = 0
        for l in range(T):
            # Calculate denominator of w_ij
            bottom += np.exp(temp[l] - max_num)
        for j in range(T):             
            #Calculate numerator of w_ij
            top = np.sum(x[i,:] * np.log(p[j,:])) + np.log(pi[j]) - max_num
            top = np.exp(top)
            w[i,j] = top/bottom
            
    for j in range(T):
        new_p_top = np.zeros(W)
        new_p_bot = np.zeros(W)
        # For each topic, for each document, recalculate new probabilities based on the previous probabilities
        for i in range(D):
            new_p_top += x[i] * w[i,j]
            new_p_bot += np.dot(x[i], np.ones(W)) * w[i,j]
        
        p[j,:] = new_p_top/new_p_bot
        
        sum_pi = 0.0
        for i in range(D):
            sum_pi += w[i,j]
        # New average pi_j
        pi[j] = sum_pi/D

In [ ]:
count = 0
while (1):
    count = count + 1
    
    old_P = np.copy(P)
    old_pi = np.copy(pi)
    
    em_step(doc_word, w, P, pi)
    
    # Check change in p and pi after EM step
    norm_P = np.linalg.norm(P-old_P)
    norm_pi = np.linalg.norm(pi-old_pi)
    
    print(count, norm_P, norm_pi)
    # Run EM until an epsilon is reached
    if norm_P < 0.0001:
        break

In [ ]:
bar_width = 1
plt.bar(np.arange(pi.shape[0])*bar_width, pi, bar_width)
plt.title('Topic vs. Probability')
plt.xlabel('Topic Number')
plt.ylabel('Probability')
plt.show()

In [ ]:
top_words = []
for t in range(T):
    # Sort words in each topic by probability
    top_ten = P[t].argsort()[-10:][::-1]
    temp_words = []
    for i in top_ten:
        # Get the top ten words
        temp_words.append(vocab[i])
    top_words.append(temp_words)


In [ ]:
print(np.array(top_words))

In [ ]:
df = pd.DataFrame(np.asarray(top_words)).T
df[[x for x in range(12)]]

In [ ]:
df[[x for x in range(12, 24)]]

In [ ]:
df[[x for x in range(24, 30)]]